In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from passivbot_futures import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
user = 'your_user_name'
settings = load_settings(user)
s = settings['symbol']
settings

In [ ]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(user, settings)
cc = bot.cc

In [ ]:
n_days_load = 2
adf = await bot.load_trades(s, n_days_load)

In [ ]:
adf.price.iloc[::100].plot()

In [ ]:
# modify settings
settings['entry_amount'] = 0.001
settings['leverage'] = 100
settings['markup'] = 0.0015
settings['flashcrash_factor'] = 0.001
settings['ema_span'] = 1000
settings['enter_long'] = True
settings['enter_shrt'] = True
settings

In [ ]:
logs, trades, adf = backtest(adf, settings)

tdf = pd.DataFrame(trades).set_index('agg_id')
adf.price.plot()
bdf = tdf[tdf.side == 'buy']
sdf = tdf[tdf.side == 'sel']
bdf.price.plot(style='bo')
sdf.price.plot(style='ro')
tdf

In [ ]:
ldf = pd.DataFrame(logs)
realized_pnl_sum = ldf.realized_pnl_sum.max()
print('realized pnl sum', realized_pnl_sum)
ldf

In [ ]:
print('highest n double downs')
ldf.n_double_downs.sort_values().tail(20)

In [ ]:
max_needed_margin = ldf.initial_margin_max.max()
print('max_needed_margin', max_needed_margin)
print('ldf.pos_amount.min()', ldf.pos_amount.min())
print('ldf.pos_amount.max()', ldf.pos_amount.max())
print('ldf.n_double_downs.max()', ldf.n_double_downs.max())

In [ ]:
ldf.realized_pnl_sum.plot()

In [ ]:
ldf.pos_amount.plot()

In [ ]:
gain = (realized_pnl_sum + max_needed_margin) / max_needed_margin
n_days = (tdf.timestamp.max() - tdf.timestamp.min()) / 1000 / 60 / 60 / 24
adg = gain ** (1 / n_days)
average_daily_abs_profit = realized_pnl_sum / n_days
print('max_needed_margin', max_needed_margin)
print('realized_pnl_sum', realized_pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average daily gain', adg)
print('average_daily_abs_profit', average_daily_abs_profit)

In [ ]:
# plot last day's trades
age_limit = (time() - 60 * 60 * 24) * 1000
adfc = adf[adf.timestamp > age_limit]
tdfc = tdf[tdf.timestamp > age_limit]

adfc.price.plot()
bdfc = tdfc[tdfc.side == 'buy']
sdfc = tdfc[tdfc.side == 'sel']
bdfc.price.plot(style='bo')
sdfc.price.plot(style='ro')